<a href="https://colab.research.google.com/github/ArezooNajafi/-SpaceX-Falcon-9-first-stage-Landing-Prediction/blob/main/Falcon_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install requests
!pip install beautifulsoup4


In [ ]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

In [ ]:
import pandas as pd
import requests
response=requests.get(static_json_url)
df=pd.json_normalize(response.json())
df.head(5)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.describe(include='all')

In [ ]:
import datetime
data = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [ ]:
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

In [ ]:
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [ ]:
BoosterVersion

In [ ]:
# Call getBoosterVersion
getBoosterVersion(data)

In [ ]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [ ]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [ ]:
BoosterVersion[0:5]

In [ ]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

In [ ]:
# Call getLaunchSite
getLaunchSite(data)

In [ ]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [ ]:
# Call getPayloadData
getPayloadData(data)

In [ ]:
PayloadData[0:5]

In [ ]:
# Call getCoreData
getCoreData(data)

In [ ]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

In [ ]:
ds=pd.DataFrame(launch_dict)

In [ ]:
for key, value in launch_dict.items():
    print(f"{key}: {len(value)}")

In [ ]:
ds.nunique()

In [ ]:
ds=ds[ds['BoosterVersion']!='Falcon 1']

In [ ]:
ds.describe()

In [ ]:
ds.isnull().sum()

In [ ]:
# Instead of:
# ds=ds['PayloadMass'].fillna(df['PayloadMass'].mean(), inplace=True)

# Use this to fill NaN values in 'PayloadMass' column of ds DataFrame:
mean=ds['PayloadMass'].mean()
ds['PayloadMass'].fillna(mean, inplace=True)

In [ ]:
ds.head()

In [ ]:
ds.isnull().sum()

In [ ]:
ds.dtypes

In [ ]:
lunchstites_num = ds.groupby('LaunchSite')['LaunchSite'].value_counts()
lunchstites_num

In [ ]:
orbit_occur=ds['Orbit'].value_counts()
orbit_occur

In [ ]:
landing_outcomes = ds['Outcome'].value_counts()
landing_outcomes

In [ ]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

In [ ]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

In [ ]:
# Instead of:
# labeld_outcome= if outcome in bad_outcomes:
#    landing_class=0
# else:
#    landing_class=1

# Use this to create a new column 'landing_class' based on the 'Outcome' column:
ds['landing_class'] = ds['Outcome'].apply(lambda outcome: 0 if outcome in bad_outcomes else 1)

In [ ]:
ds.head(5)

In [ ]:
ds[['landing_class']].head(8)

In [ ]:
ds['landing_class'].mean()

In [ ]:
ds['landing_class'].value_counts()

In [ ]:
ds[ds['Orbit'].isin(['GEO', 'GTO'])].shape[0]

In [ ]:
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns

In [ ]:
import requests
import io

URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
response = requests.get(URL)
response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
dataset_part_2_csv = io.BytesIO(response.content)  # Use response.content instead of response.text for binary data
tbl = pd.read_csv(dataset_part_2_csv)
tbl.head(5)

In [ ]:
sns.catplot(y="PayloadMass", x="FlightNumber", hue="Class", data=tbl, aspect = 5)
plt.xlabel("Flight Number",fontsize=20)
plt.ylabel("Pay load Mass (kg)",fontsize=20)
plt.show()
#

In [ ]:
sns.catplot(data=tbl,x='PayloadMass',y='LaunchSite',hue='Class',aspect=1)
plt.xlabel("Pay load Mass (kg)",fontsize=20)
plt.ylabel("LaunchSite",fontsize=20)
plt.show()
#

In [ ]:
sns.catplot(data=tbl,x='PayloadMass',y='Orbit',hue='Class',aspect=1)
plt.xlabel("Pay load Mass (kg)",fontsize=20)
plt.ylabel("Orbit",fontsize=20)
plt.show()

In [ ]:
# A function to Extract years from the date
year=[]
def Extract_year():
    for i in tbl["Date"]:
        year.append(i.split("-")[0])
    return year
Extract_year()
tbl['Date'] = year
tbl.head()


In [ ]:
succes_rate=tbl[tbl['Class']==1]['LaunchSite'].value_counts()/tbl['LaunchSite'].value_counts()
succes_rate

In [ ]:
yearly_sucess=tbl.groupby('Date')['Class'].mean()
yearly_sucess

In [ ]:
sns.lineplot(data=yearly_sucess)
plt.xlabel("Year",fontsize=20)
plt.ylabel("Success Rate",fontsize=20)
plt.show()

In [ ]:
features = tbl[['FlightNumber', 'PayloadMass', 'Orbit', 'LaunchSite', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial']]
features.head()

In [ ]:
features_cleaned=pd.get_dummies(features, columns=['Orbit', 'LaunchSite', 'LandingPad','Serial'])
features_cleaned.head()

In [ ]:
features_cleaned=features_cleaned.astype('float64')
features_cleaned.head()

In [ ]:
import folium

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
m = folium.Map(location=[37.7749, -122.4194], zoom_start=6)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(m)

# Iterate through the DataFrame and add markers to the cluster
for index, row in tbl.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['LaunchSite']
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save('launch_sites_map.html')
m

In [ ]:
!python3.11 -m pip install pandas dash

In [ ]:
  options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]

In [ ]:
  dcc.Dropdown(id='id',
                options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'site1', 'value': 'site1'},
                ],
                value='ALL',
                placeholder="place holder here",
                searchable=True
                ),

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
app = dash.Dash(__name__)

# Define the layout of the application
app.layout = html.Div([
    html.H1("Interactive Dashboard for Launch Sites"),

    # Dropdown for selecting LaunchSite
    dcc.Dropdown(
        id='launchsite-dropdown',
        options=[{'label': site, 'value': site} for site in tbl['LaunchSite']],
        value='Site A',  # Default value
        style={'width': '50%'}
    ),

    # Graph to display Price vs. Quantity based on LaunchSite selection
    dcc.Graph(id='price-quantity-graph'),

    # Optional: Adding a map to visualize launch site locations on the map
    dcc.Graph(id='launchsite-map')
])

# Define the callback to update the graph
@app.callback(
    [Output('price-quantity-graph', 'figure'),
     Output('launchsite-map', 'figure')],
    [Input('launchsite-dropdown', 'value')]
)
def update_dashboard(selected_site):
    # Filter the DataFrame based on the selected launch site
    filtered_df = tbl[tbl['LaunchSite'] == selected_site]

    # Create a bar chart showing Price vs. Quantity
    price_quantity_fig = px.bar(filtered_df, x='LaunchSite', y=['Price', 'Quantity'],
                                title=f'Price and Quantity of {selected_site}')

    # Create a scatter mapbox to visualize the launch site on a map
    map_fig = px.scatter_mapbox(filtered_df, lat='Latitude', lon='Longitude',
                                hover_name='LaunchSite', size='Quantity',
                                title=f'Launch Site Location: {selected_site}')

    map_fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=5, mapbox_center={"lat": filtered_df['Latitude'].iloc[0], "lon": filtered_df['Longitude'].iloc[0]})

    return price_quantity_fig, map_fig

# Run the Dash app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px



# Create a list of unique launch sites
launch_sites = [{'label': 'All Sites', 'value': 'ALL'}] + \
               [{'label': site, 'value': site} for site in tbl['LaunchSite'].unique()]

# Create the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    dcc.Dropdown(
        id='site-dropdown',
        options=launch_sites,
        value='ALL',
        placeholder="Select a Launch Site",
        searchable=True
    ),

    html.Br(),

    html.Div(dcc.Graph(id='success-rate-chart')),

    html.Br(),

    html.Div(dcc.Graph(id='launchsite-map'))
])

# Callback to update charts based on selected site
@app.callback(
    [Output(component_id='success-rate-chart', component_property='figure'),
     Output(component_id='launchsite-map', component_property='figure')],
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_dashboard(selected_site):
    # Filter the dataframe based on the selected site
    if selected_site == 'ALL':
        filtered_tbl = tbl
    else:
        filtered_tbl = tbl[tbl['LaunchSite'] == selected_site]

    # Compute success rate (Class column with capital 'C')
    success_rate = filtered_tbl['Class'].mean()

    # Success rate pie chart
    success_fig = px.pie(
        names=['Success', 'Failure'],
        values=[success_rate, 1 - success_rate],
        title=f"Launch Success Rate for {selected_site}"
    )

    # Map showing launch locations
    map_fig = px.scatter_mapbox(
        filtered_tbl,
        lat="Latitude",
        lon="Longitude",
        hover_name="LaunchSite",
        hover_data=["PayloadMass", "Class"],
        color="Class",
        zoom=3,
        height=500
    )
    map_fig.update_layout(mapbox_style="open-street-map")
    map_fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    return success_fig, map_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# Convert the 'Class' column into a NumPy array and assign it to variable Y
Y = tbl['Class'].to_numpy()

# Ensure that Y is a Pandas Series (in this case it is still a NumPy array but you can access it as a Pandas Series)
print(type(Y))  # This will print <class 'numpy.ndarray'> because it's a NumPy array


In [ ]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
# Preprocessing allows us to standarsize our data
from sklearn import preprocessing
# Allows us to split our data into training and testing data
from sklearn.model_selection import train_test_split
# Allows us to test parameters of classification algorithms and find the best one
from sklearn.model_selection import GridSearchCV
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression
# Support Vector Machine classification algorithm
from sklearn.svm import SVC
# Decision Tree classification algorithm
from sklearn.tree import DecisionTreeClassifier
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# students get this
transform = preprocessing.StandardScaler()

In [ ]:
X=features_cleaned
X=transform.fit_transform(X)
Y=tbl['Class']

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [ ]:
X_test.shape

In [ ]:
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

In [ ]:
parameters ={"C":[0.01,0.1,1],'penalty':['l2'], 'solver':['lbfgs']}# l1 lasso l2 ridge
lr=LogisticRegression()


In [ ]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# ... (other imports)

# Define your parameters for GridSearchCV
parameters = {"C": [0.01, 0.1, 1], 'penalty': ['l2'], 'solver': ['lbfgs']}

# Create a LogisticRegression object
lr = LogisticRegression()

# Create a GridSearchCV object and fit it to your training data
logreg_cv = GridSearchCV(lr, parameters, cv=10)  # Assuming you want 10-fold cross-validation
logreg_cv.fit(X_train, Y_train)

# Now you can access the best parameters and score
print("tuned hpyerparameters :(best parameters) ", logreg_cv.best_params_)
print("accuracy :", logreg_cv.best_score_)

In [ ]:
def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix');
    ax.xaxis.set_ticklabels(['did not land', 'land']); ax.yaxis.set_ticklabels(['did not land', 'landed'])
    plt.show()

In [ ]:
yhat=logreg_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

In [ ]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC  # Import SVC

# ... (other code) ...

# Create the SVC object
svm = SVC()

# Define the parameter grid for GridSearchCV
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}

# Create the GridSearchCV object and fit it
svm_cv = GridSearchCV(svm, parameters, cv=10)  # Create svm_cv here
svm_cv.fit(X_train, Y_train)  # Fit the model

# Now you can print the results
print("tuned hpyerparameters :(best parameters) ",svm_cv.best_params_)
print("accuracy :",svm_cv.best_score_)

In [ ]:
best_kernel = svm_cv.best_params_['kernel']
print(f"The best kernel is: {best_kernel}")


In [ ]:
yhat=svm_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

In [ ]:
# ... (previous code) ...

# Create the GridSearchCV object and fit it
svm_cv = GridSearchCV(svm, parameters, cv=10)
svm_cv.fit(X_train, Y_train)

# Calculate accuracy on the test data
test_accuracy = svm_cv.score(X_test, Y_test)

# Print the test accuracy
print(f"Accuracy on test data: {test_accuracy}")

In [ ]:
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

tree = DecisionTreeClassifier()

In [ ]:
tree_cv = GridSearchCV(tree, parameters, cv=10)
tree_cv.fit(X_train, Y_train)

In [ ]:
yhat = tree_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

In [ ]:
test_accuracy = tree_cv.score(X_test, Y_test)
print(f"Accuracy on test data: {test_accuracy}")

In [ ]:
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

KNN = KNeighborsClassifier()

In [ ]:
KNN= GridSearchCV(KNN, parameters, cv=10)
KNN.fit(X_train, Y_train)

In [ ]:
yhat = KNN.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

In [ ]:
test_accuracy = KNN.score(X_test, Y_test)
print(f"Accuracy on test data: {test_accuracy}")

In [ ]:
!pip install ipython-sql
!pip install ipython-sql prettytable

In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
%sql sqlite:///my_data1.db

In [ ]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

In [ ]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

In [ ]:
# Step 1: Install ipython-sql (only needed once)
!pip install ipython-sql

# Step 2: Load ipython-sql
%load_ext sql

# Step 3: Connect to your SQLite database (replace with your actual file path if needed)
%sql sqlite:///spacex_data.db

# Step 4: Run your SQL query
x = %sql SELECT DISTINCT Launch_Site FROM SPACEXTABLE;
print(x)


In [ ]:
# Create new table from cleaned data
%%sql
CREATE TABLE SPACEXTABLE AS
SELECT * FROM SPACEXTBL
WHERE Date IS NOT NULL;


In [ ]:
# Step 1: Import libraries and connect to database
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'

# Connect to SQLite DB
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Step 2: Load CSV and store in SQLite table
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# Step 3: Drop existing table if any, and create a clean version
cur.execute("DROP TABLE IF EXISTS SPACEXTABLE;")
cur.execute("""
    CREATE TABLE SPACEXTABLE AS
    SELECT * FROM SPACEXTBL;
""")
con.commit()
# Step 1: Import libraries and connect to database
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'

# Connect to SQLite DB
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Step 2: Load CSV and store in SQLite table
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# Step 3: Drop existing table if any, and create a clean version
cur.execute("DROP TABLE IF EXISTS SPACEXTABLE;")
cur.execute("""
    CREATE TABLE SPACEXTABLE AS
    SELECT * FROM SPACEXTBL;
""")
con.commit()

# Step 4: Query distinct launch sites
result = cur.execute("SELECT \"Launch Site\" FROM SPACEXTABLE;") #added quotes
rows = result.fetchall()

# Step 5: Display results
from prettytable import from_db_cursor
result = cur.execute("SELECT DISTINCT \"Launch Site\" FROM SPACEXTABLE;") #added quotes and DISTINCT
table = from_db_cursor(result)
print(table)
# Step 1: Import libraries and connect to database
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'

# Connect to SQLite DB
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Step 2: Load CSV and store in SQLite table
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# Step 3: Drop existing table if any, and create a clean version
cur.execute("DROP TABLE IF EXISTS SPACEXTABLE;")
cur.execute("""
    CREATE TABLE SPACEXTABLE AS
    SELECT * FROM SPACEXTBL;
""")
con.commit()
# Step 1: Import libraries and connect to database
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'

# Connect to SQLite DB
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Step 2: Load CSV and store in SQLite table
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# Step 3: Drop existing table if any, and create a clean version
cur.execute("DROP TABLE IF EXISTS SPACEXTABLE;")
cur.execute("""
    CREATE TABLE SPACEXTABLE AS
    SELECT * FROM SPACEXTBL;
""")
con.commit()

# Step 4: Query distinct launch sites
result = cur.execute("SELECT \"Launch Site\" FROM SPACEXTABLE;") #added quotes
rows = result.fetchall()





In [ ]:
# Step 4: Query distinct launch sites
result = cur.execute("SELECT \"Launch Site\" FROM SPACEXTABLE WHERE \"Launch Site\" LIKE 'CCA%' LIMIT 5;")
rows = result.fetchall()

# Print the result
for row in rows:
    print(row)
